## libraries

In [ ]:
import tensorflow as tf
import keras
from keras.datasets import mnist, fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from functools import partial

## initial settings

In [ ]:
num_classes = 10 #defined from datasets
epochs = 10
validation_size = 0.1

## define functions

### permute data

In [ ]:
def permuate_data(x):
    for i in range(len(x)):
        x[i] = np.random.permutation(x[i])
        for j in range(len(x[i])):
            x[i][j] = np.random.permutation(x[i][j])
    return x

### train test split

In [ ]:
def splitData(dataset):
  (X_train, y_train), (X_test, y_test) = dataset.load_data()

  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_size)

  X_train = X_train.reshape(int(60000 * (1.0 - validation_size)), 28, 28, 1)
  X_test = X_test.reshape(10000, 28, 28, 1)
  X_valid = X_valid.reshape(int(60000 * validation_size), 28, 28, 1)

  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_valid = X_valid.astype('float32')

  X_train /= 255
  X_test /= 255
  X_valid /= 255

  # convert class vectors to binary class matrices
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)
  y_valid = keras.utils.to_categorical(y_valid, num_classes)


  X_train = permuate_data(X_train)
  X_test = permuate_data(X_test)
  X_valid = permuate_data(X_valid)

  return X_train, y_train, X_test, y_test, X_valid, y_valid

### train model

In [ ]:
def trainModel(model, X_train, y_train, X_valid, y_valid, optim='sgd'):
    model.compile(loss="categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"])
    
    #track epoch loss history
    history = model.fit(X_train, y_train, epochs=epochs,
                        validation_data=(X_valid, y_valid))
    
    return history

### plot loss over epochs

In [ ]:
def pltHistory(history):
  pd.DataFrame(history.history).plot(figsize=(8, 5))
  plt.grid(True)
  plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
  plt.title("Loss and Accuracy over Epochs")
  plt.xlabel('Epochs')
  plt.yablel('Loss/Accuracy')
  plt.show()

### execute experiments for a given model and dataset

In [ ]:
def doExperiment(model, dataset, plot_hist=True):
  X_train, y_train, X_test, y_test, X_valid, y_valid = splitData(dataset)
  # print("--- Training Model ...")
  history = trainModel(model, X_train, y_train, X_valid, y_valid)
  if plot_hist == True:
    print("\n\n ========= History =========")
    pltHistory(history)
  print("\n\n ========= Evaluation (Loss, Accuracy) ========= ")
  return model.evaluate(X_test, y_test)

## mlp

In [ ]:
mlp_model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=[28, 28, 1]),
  keras.layers.Dense(300, activation="relu"),
  keras.layers.Dense(100, activation="relu"),
  keras.layers.Dense(10, activation="softmax")
])

### mnist dataset

In [ ]:
doExperiment(mlp_model, mnist, plot_hist=True)

### fashion mnist

In [ ]:
doExperiment(mlp_model, fashion_mnist, plot_hist=True)

## cnn 

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, activation='relu', padding="SAME")

cnn_model = keras.models.Sequential([
DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
keras.layers.MaxPooling2D(pool_size=2),
DefaultConv2D(filters=128),
DefaultConv2D(filters=128),
keras.layers.MaxPooling2D(pool_size=2),
DefaultConv2D(filters=256),
DefaultConv2D(filters=256),
keras.layers.MaxPooling2D(pool_size=2),
keras.layers.Flatten(),
keras.layers.Dense(units=128, activation='relu'),
keras.layers.Dropout(0.5),
keras.layers.Dense(units=64, activation='relu'),
keras.layers.Dropout(0.5),
keras.layers.Dense(units=10, activation='softmax'),
])

### mnist

In [ ]:
doExperiment(cnn_model, mnist)

### fashion mnist

In [ ]:
doExperiment(cnn_model, fashion_mnist)